In [1]:
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.research import run_pipeline
from quantopian.pipeline.factors import CustomFactor, Returns, SimpleBeta
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline import Pipeline
import pandas as pd

define custom factors

In [2]:
class CrossSectionalMomentum(CustomFactor):    
    inputs = [USEquityPricing.close]
    window_length = 252
    def compute(self, today, assets, out, prices):
        prices = pd.DataFrame(prices)
        R = (prices / prices.shift(100))
        out[:] = (R.T - R.T.mean()).T.mean()

In [10]:
def make_pipeline():
    #universe
    initial_screen=QTradableStocksUS()
    
    #use custom factors
    cross_momentum = CrossSectionalMomentum()
    
    #use built in factors
    abs_momentum = Returns(inputs=[USEquityPricing.close], window_length=252,mask=initial_screen)
    beta=SimpleBeta(target=symbols(8554),regression_length=100)
    
    #use factors from Fundamentals data set
    pb=Fundamentals.pb_ratio.latest
    
    #rank factors
    cross_momentum_rank=cross_momentum.rank(mask=initial_screen)
    abs_momentum_rank=abs_momentum.rank(mask=initial_screen)
    
    #combine factors
    combined_rank = (cross_momentum_rank+ abs_momentum_rank)
    
    #select assets according to factors
    longs = combined_rank.percentile_between(95, 100)
    shorts = combined_rank.percentile_between(0, 5)

    #setup final screen
    pipe_screen = (combined_rank.notnan()&beta.notnan()&initial_screen&pb.notnan())
    
    #define columns
    pipe_columns = {
        'long':longs,
        'short': shorts,
        'pb':pb,
        'combined_rank':combined_rank,
        'beta':beta
    }
    pipe = Pipeline(columns = pipe_columns, screen = pipe_screen)
    return pipe

run pipeline

In [13]:
output=run_pipeline(make_pipeline(),start_date='01/01/2017',end_date='12/31/2017')
output.head()

beta  combined_rank   long  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])   2.605633          368.0  False   
                          Equity(24 [AAPL])  0.785678         1752.0  False   
                          Equity(31 [ABAX])  1.460706         1478.0  False   
                          Equity(41 [ARCB])  1.693973         2793.0  False   
                          Equity(52 [ABM])   1.216629         3270.0  False   

                                                   pb  short  
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])   0.693811  False  
                          Equity(24 [AAPL])  4.815542  False  
                          Equity(31 [ABAX])  4.697926  False  
                          Equity(41 [ARCB])  1.190387  False  
                          Equity(52 [ABM])   2.329462  False